## Przykłady

In [2]:
from auth import openai_api_key

In [4]:
# 01_langchain_init

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

# Inicjalizacja domyślnego modelu, czyli gpt-3.5-turbo
chat = ChatOpenAI(openai_api_key=openai_api_key)

# Wywołanie modelu poprzez przesłanie tablicy wiadomości.
# W tym przypadku to proste przywitanie
message = HumanMessage("Hej, jestem Justynka!")
response = chat.invoke([message])

print(response.content)

Cześć Justynko! Jak się masz? W czym mogę Ci dzisiaj pomóc?


In [25]:
# 02_langchain_format

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
# from langchain.context import context

# Treść kontekstu
context = """
Vercel AI SDK to biblioteka typu open-source zaprojektowana, aby pomóc programistom tworzyć interfejsy użytkownika dla konwersacji, streamingu i czatu w JavaScript i TypeScript. SDK wspiera React/Next.js, Svelte/SvelteKit, przy czym wsparcie dla Nuxt/Vue będzie dostępne wkrótce.
Aby zainstalować SDK, wprowadź następujące polecenie w terminalu:
npm install ai
"""

# Zwykle do definiowania promptów warto korzystać z template strings
# Tutaj treści zamknięte w klamrach {} są zastępowane przez LangChain konkretnymi wartościami
system_template = """
As a {role} who answers the questions ultra-concisely using CONTEXT below 
and nothing more and truthfully says "don't know" when the CONTEXT is not enough to give an answer.

context###{context}###
"""

human_template = "{text}"

# Lista wiadomości
messages = [
    ("system", system_template),
    ("human", human_template)
]

# Inicjalizacja promptu
chat_prompt = ChatPromptTemplate.from_messages(messages)

# Faktyczne uzupełnienie szablonów wartościami
formatted_chat_prompt = chat_prompt.format_messages(
    context= context,
    role= "Doświadczony programista JavaScript",
    text= "Co to wyrażenia lambda w Pythonie?",
)

# Inicjalizacja domyślnego modelu, czyli gpt-3.5-turbo
chat = ChatOpenAI(openai_api_key=openai_api_key)

# Wykonanie zapytania do modelu
response =  chat.invoke(formatted_chat_prompt)

print(response.content)

Nie wiem


W przypadku definiowania contextu i systemowej templatki należy używać języka ANGIELSKIEGO. Jeśli robię to po polsku, to odpowiedzi są niezgodne z poleceniem - AI nie używa odpowiedzi "nie wiem". Po zdefiniowaniu wytycznych po angielsku można zadawac pytanie zarówno po angielsku jak i po polsku i odpowiada dobrze

In [18]:
# 03_langchain_stream

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

## Inicjalizacja chatu z włączonym streamingiem
chat = ChatOpenAI(
    openai_api_key=openai_api_key, 
    streaming=True
)

# Wywołanie chatu wraz z funkcją przyjmującą kolejne tokeny składające się na wypowiedź modelu
chunks = []
async for chunk in chat.astream("Cześć, jestem Justynka!"):
    chunks.append(chunk)
    print(chunk.content, end="|", flush=True)

|C|ze|ść| Just|yn|ko|!| Jak| mog|ę| Ci| dz|isia|j| pom|ó|c|?||

więcej o streamingu tutaj: https://python.langchain.com/docs/expression_language/streaming

## Zadania praktyczne

In [40]:
from utilities.task_service import get_authentication_token, get_task, send_answer
from auth import apikey

In [41]:
from dotenv import load_dotenv

# Automatycznie wyszukuje plik .env w katalogu bieżącym i załaduje zmienne środowiskowe
load_dotenv()

import os
openai_api_key = os.getenv("OPENAI_API_KEY")

### Moderation

In [42]:
# autoryzacja

task_name = "moderation"

token = get_authentication_token(apikey, task_name)
if token:
    print("Otrzymano token:", token)
else:
    print("Nie udało się uzyskać tokenu.")

Otrzymano token: 8aaef3d04af75f29ce34722db8627efff495a4f8


In [43]:
# pobranie danych do zadania

response = get_task(token)
if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania zapytania.")

Odpowiedź z serwera: {'code': 0, 'msg': 'please return 0/1 for each sentence to classify if this is content that requires moderation', 'input': ['azjaci są głupi i brzydcy i nie powinni żyć', 'ten gość musi zginąć. Nie pozwole sobię na obrażanie mnie.', 'Sasha.Grey.s3x.p0rn.extreme-interracial.S03E12.DVDRip.mp4', 'majonez Winiary jest lepszy od Kieleckiego']}


In [44]:
print(response['input'])

['azjaci są głupi i brzydcy i nie powinni żyć', 'ten gość musi zginąć. Nie pozwole sobię na obrażanie mnie.', 'Sasha.Grey.s3x.p0rn.extreme-interracial.S03E12.DVDRip.mp4', 'majonez Winiary jest lepszy od Kieleckiego']


In [45]:
from openai import OpenAI
client = OpenAI()

answers = []
for statement in response['input']:
    response = client.moderations.create(input=statement)
    
    output = response.results[0]
    if(output.flagged) == True:
        answers.append(1)
    else:
        answers.append(0)

In [46]:
print(answers)

[1, 1, 1, 0]


In [47]:
# wysłanie odpowiedzi

response = send_answer(token, answers)
if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania odpowiedzi.")

Odpowiedź z serwera: {'code': 0, 'msg': 'OK', 'note': 'CORRECT'}


### blogger

In [48]:
# autoryzacja

task_name = "blogger"

token = get_authentication_token(apikey, task_name)
if token:
    print("Otrzymano token:", token)
else:
    print("Nie udało się uzyskać tokenu.")

Otrzymano token: 26835f56f166a7639cb6ed45f71caf6ff18f71be


In [49]:
# pobranie danych do zadania

response = get_task(token)
if response:
    print("Odpowiedź z serwera:", response)
else:
    print("Wystąpił błąd podczas wysyłania zapytania.")

Odpowiedź z serwera: {'code': 0, 'msg': 'please write blog post for the provided outline', 'blog': ['Wstęp: kilka słów na temat historii pizzy', 'Niezbędne składniki na pizzę', 'Robienie pizzy', 'Pieczenie pizzy w piekarniku']}


In [50]:
print(response['blog'])

['Wstęp: kilka słów na temat historii pizzy', 'Niezbędne składniki na pizzę', 'Robienie pizzy', 'Pieczenie pizzy w piekarniku']
